### Import Library

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import joblib


In [2]:
df = pd.read_csv('wisata.csv')
df

,id,nama,kategori,lokasi,deskripsi
0,1,Danau Toba,Alam,Sumatera Utara,Danau vulkanik terbesar di Indonesia dengan bu...
1,2,Borobudur,Budaya,Jawa Tengah,Candi Buddha terbesar di dunia dan situs waris...
2,3,Raja Ampat,Alam,Papua Barat,Kepulauan dengan keanekaragaman laut yang luar...
3,4,Bromo,Alam,Jawa Timur,Gunung berapi aktif dengan pemandangan matahar...
4,5,Ubud,Budaya,Bali,Pusat seni dan budaya Bali dengan pemandangan ...
5,6,Tanah Lot,Religi,Bali,Pura ikonik di atas batu karang di tengah laut
6,7,Kawah Ijen,Alam,Jawa Timur,Gunung dengan api biru dan danau asam tertingg...
7,8,Toba Samosir,Budaya,Sumatera Utara,Wilayah dengan tradisi Batak yang kuat di seki...
8,9,Derawan,Alam,Kalimantan Timur,"Pulau eksotis dengan penyu, ubur-ubur, dan pan..."
9,10,Leang-Leang,Sejarah,Sulawesi Selatan,Goa purba dengan lukisan tangan manusia ribuan...


In [3]:
# === 3. GABUNGKAN FITUR TEXT ===
df["combined"] = (
    df["nama"].fillna("") + " " +
    df["kategori"].fillna("") + " " +
    df["lokasi"].fillna("") + " " +
    df["deskripsi"].fillna("")
)


In [4]:
# === 4. STOPWORDS BAHASA INDONESIA ===
stop_words_indonesia = [
    "yang", "dan", "di", "ke", "dari", "pada", "adalah", "dengan", "ini", "itu",
    "untuk", "sebagai", "juga", "akan", "oleh", "karena", "maka", "tidak", "dalam",
    "sebuah", "atau", "terbesar", "terkenal", "tempat", "wisata", "pulau", "kota"
]

In [5]:

# === 5. TF-IDF VECTORIZER ===
vectorizer = TfidfVectorizer(stop_words=stop_words_indonesia)
tfidf_matrix = vectorizer.fit_transform(df["combined"])

In [6]:

# === 6. HITUNG SIMILARITAS COSINE ===
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [7]:
# === 7. SIMPAN SEMUA KE SATU FILE ===
model_wisata = {
    "data": df,
    "vectorizer": vectorizer,
    "cosine_sim": cosine_sim
}

joblib.dump(model_wisata, "model_wisata.h5")
print("\n✅ Semua model dan data berhasil disimpan ke satu file: model_wisata.pkl")




✅ Semua model dan data berhasil disimpan ke satu file: model_wisata.pkl


In [8]:

# === 8. FUNGSI REKOMENDASI LANGSUNG DI FILE INI ===
def rekomendasi_wisata(nama_wisata: str, top_n: int = 5):
    nama_wisata = nama_wisata.lower()
    if nama_wisata not in df['nama'].str.lower().values:
        print("❌ Wisata tidak ditemukan.")
        return []

    idx = df[df['nama'].str.lower() == nama_wisata].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:top_n+1]

    rekomendasi = []
    for i, (rec_idx, score) in enumerate(sim_scores):
        row = df.iloc[rec_idx]
        rekomendasi.append({
            "nama": row["nama"],
            "lokasi": row["lokasi"],
            "kategori": row["kategori"],
            "score": round(score, 3)
        })

    return rekomendasi


In [16]:
rekomendasi_wisata("Raja Ampat")

[{'nama': 'Karimunjawa',
  'lokasi': 'Jawa Tengah',
  'kategori': 'Alam',
  'score': 0.248},
 {'nama': 'Lombok',
  'lokasi': 'Nusa Tenggara Barat',
  'kategori': 'Alam',
  'score': 0.102},
 {'nama': 'Puncak',
  'lokasi': 'Jawa Barat',
  'kategori': 'Alam',
  'score': 0.098},
 {'nama': 'Pulau Komodo',
  'lokasi': 'Nusa Tenggara Timur',
  'kategori': 'Alam',
  'score': 0.085},
 {'nama': 'Bukittinggi',
  'lokasi': 'Sumatera Barat',
  'kategori': 'Budaya',
  'score': 0.064}]